https://github.com/JuliaStats/StatsModels.jl/issues/220

In [1]:
using DataFrames
using GLM
using StatsBase: sample

n = 20
r = 10
x_symbols = [Symbol("x$i") for i in 1:n]
@time df = DataFrame(rand(100, n+1), [:y; x_symbols]);

  0.533677 seconds (1.89 M allocations: 108.855 MiB, 21.78% gc time, 99.95% compilation time)


In [2]:
x_vars = sample(x_symbols, r; replace=false)
@time M = term(:y) ~ sum(term(x) for x in x_vars)
@time cols = Tables.columntable(df)
@time mf = ModelFrame(M, cols, model=LinearModel)
@time mm = ModelMatrix(mf)
@time y = response(mf)
@time result_fit = fit(LinearModel, mm.m, y)
@time regmodel = StatsModels.TableRegressionModel(result_fit, mf, mm)

  0.147477 seconds (279.83 k allocations: 18.107 MiB, 41.70% gc time, 100.88% compilation time)
  0.224159 seconds (653.37 k allocations: 41.294 MiB, 3.31% gc time, 44.19% compilation time)
  1.547989 seconds (5.92 M allocations: 371.880 MiB, 5.01% gc time)
  0.301086 seconds (1.19 M allocations: 75.524 MiB, 5.58% gc time)
  0.005957 seconds (3.78 k allocations: 283.598 KiB, 99.80% compilation time)
  1.168012 seconds (5.13 M allocations: 290.797 MiB, 6.93% gc time, 0.11% compilation time)
  0.003617 seconds (3.76 k allocations: 276.488 KiB, 96.63% compilation time)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x5 + x13 + x19 + x3 + x12 + x8 + x10 + x2 + x9 + x11

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)  Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.667177     0.162083    4.12    <1e-04   0.345122   0.989233
x5            0.0723408    0.0990105   0.73    0.4669  -0.124391   0.269072
x13          -0.0527063    0.104441   -0.50    0.6150  -0.260228   0.154815
x19          -0.0619292    0.0991991  -0.62    0.5340  -0.259036   0.135177
x3            0.0943942    0.106745    0.88    0.3789  -0.117706   0.306495
x12          -0.112603     0.110635   -1.02    0.3115  -0.332432   0.107226
x8           -0.202303     0.0989246  -2.05    0.0438  -

In [3]:
x_vars = sample(x_symbols, r; replace=false)
@time M = term(:y) ~ sum(term(x) for x in x_vars)
@time cols = Tables.columntable(df)
@time mf = ModelFrame(M, cols, model=LinearModel)
@time mm = ModelMatrix(mf)
@time y = response(mf)
@time result_fit = fit(LinearModel, mm.m, y)
@time regmodel = StatsModels.TableRegressionModel(result_fit, mf, mm)

  0.074074 seconds (79.69 k allocations: 5.129 MiB, 99.54% compilation time)
  0.000023 seconds (29 allocations: 1.766 KiB)
  0.056590 seconds (136.59 k allocations: 8.867 MiB, 7.10% compilation time)
  0.063102 seconds (80.33 k allocations: 5.021 MiB, 99.69% compilation time)
  0.006432 seconds (3.77 k allocations: 282.848 KiB, 99.79% compilation time)
  0.000027 seconds (21 allocations: 23.188 KiB)
  0.003288 seconds (2.75 k allocations: 194.801 KiB, 96.37% compilation time)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x15 + x5 + x9 + x12 + x20 + x10 + x13 + x14 + x2 + x3

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   0.58099     0.16576     3.51    0.0007   0.251629   0.91035
x15           0.0751819   0.106723    0.70    0.4830  -0.136874   0.287237
x5            0.0274317   0.0966737   0.28    0.7773  -0.164657   0.21952
x9            0.158785    0.109298    1.45    0.1498  -0.0583888  0.375959
x12          -0.100501    0.110216   -0.91    0.3643  -0.319497   0.118495
x20          -0.153889    0.104357   -1.47    0.1438  -0.361243   0.0534658
x10          -0.150822    0.10306    -1.46    0.1469  -0.355601   0.

In [4]:
using DataFrames
using GLM
using StatsBase: sample

n = 20
r = 10
x_symbols = [Symbol("x$i") for i in 1:n]
df = DataFrame(rand(100, n+1), [:y; x_symbols]);

result = []
for _ in 1:10
    x_vars = sample(x_symbols, r; replace=false)
    F = term(:y) ~ sum(term(x) for x in x_vars)
    @time regmodel = lm(F, df)
    push!(result, regmodel)
end
result

  1.882571 seconds (6.59 M allocations: 447.619 MiB, 14.32% gc time)
  0.122462 seconds (223.53 k allocations: 14.365 MiB, 3.32% compilation time)
  0.131134 seconds (223.53 k allocations: 14.366 MiB, 2.86% compilation time)
  0.134729 seconds (223.53 k allocations: 14.366 MiB, 5.51% gc time, 3.13% compilation time)
  0.122915 seconds (223.54 k allocations: 14.377 MiB, 3.06% compilation time)
  0.125031 seconds (223.54 k allocations: 14.368 MiB, 3.06% compilation time)
  0.131954 seconds (223.54 k allocations: 14.372 MiB, 5.17% gc time, 3.16% compilation time)
  0.125828 seconds (223.53 k allocations: 14.365 MiB, 2.98% compilation time)
  0.123483 seconds (223.54 k allocations: 14.368 MiB, 3.09% compilation time)
  0.131207 seconds (223.53 k allocations: 14.372 MiB, 5.12% gc time, 3.15% compilation time)


10-element Vector{Any}:
 StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x20 + x6 + x12 + x9 + x3 + x5 + x13 + x7 + x17 + x11

Coefficients:
────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.670418      0.202498   3.31    0.0013   0.268059   1.07278
x20          -0.0810306     0.104993  -0.77    0.4423  -0.28965    0.127588
x6            0.112718      0.105235   1.07    0.2870  -0.0963809  0.321817
x12           0.00426084    0.102122   0.04    0.9668  -0.198653   0.207175
x9            0.0140792     0.114835   0.12    0.9027  -0.214095   0.242254
x3           -0.187993      0.1072    -1.75    0.0829  -0.400997   0.0250108
x5            0.0843542     0.1074

In [5]:
using DataFrames
using GLM
using StatsBase: sample

struct MyLinearModel{T, Y, X}
    linmodel::T
    y_var::Y
    x_vars::X
end

function my_lm(y_var, x_vars, df)
    y = df[!, y_var]
    X = [ones(nrow(df)) Matrix(df[!, x_vars])]
    linmodel = lm(X, y)
    MyLinearModel(linmodel, y_var, x_vars)
end

function Base.show(io::IO, mylm::MyLinearModel)
    linmodel, y_var, x_vars = getfield.(Ref(mylm), (:linmodel, :y_var, :x_vars))
    ct = coeftable(linmodel)
    ct.rownms .= string.((Symbol("(Intercept)"), x_vars...,))
    print(io, "Formula: ")
    print(io, y_var, " ~ 1")
    for x in x_vars print(io, " + ", x) end
    print(io, "\n\n")
    show(io, ct)
    print(io, "\n")
end

n = 20
r = 10
x_symbols = [Symbol("x$i") for i in 1:n]
df = DataFrame(rand(100, n+1), [:y; x_symbols]);

myresult = []
for _ in 1:10
    x_vars = sample(x_symbols, r; replace=false)
    @time mylinmodel = my_lm(:y, x_vars, df)
    push!(myresult, mylinmodel)
end
myresult

  0.246865 seconds (604.40 k allocations: 37.423 MiB, 3.17% gc time)
  0.000039 seconds (56 allocations: 43.516 KiB)
  0.000025 seconds (56 allocations: 43.516 KiB)
  0.000020 seconds (56 allocations: 43.516 KiB)
  0.000019 seconds (56 allocations: 43.516 KiB)
  0.000024 seconds (56 allocations: 43.516 KiB)
  0.000037 seconds (56 allocations: 43.516 KiB)
  0.000023 seconds (56 allocations: 43.516 KiB)
  0.000089 seconds (56 allocations: 43.516 KiB)
  0.000039 seconds (56 allocations: 43.516 KiB)


10-element Vector{Any}:
 Formula: y ~ 1 + x8 + x5 + x9 + x13 + x20 + x12 + x1 + x18 + x17 + x11

───────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   0.523086      0.184343   2.84    0.0056   0.1568    0.889372
x8           -0.187771      0.110328  -1.70    0.0923  -0.406991  0.0314491
x5            0.063159      0.110969   0.57    0.5707  -0.157335  0.283653
x9            0.105028      0.112608   0.93    0.3535  -0.11872   0.328777
x13           0.0831336     0.118765   0.70    0.4858  -0.152849  0.319116
x20           0.131395      0.118247   1.11    0.2695  -0.10356   0.366349
x12          -0.0613816     0.122774  -0.50    0.6183  -0.305331  0.182568
x1           -0.0427174     0.101262  -0.42    0.6742  -0.243923  0.158489
x18          -0.0526129     0.108177  -0.49    0.6279  -0.267559  0.162333